In [3]:
import joblib
import pandas as pd

In [2]:
forecast = joblib.load('models/310A_DI_3302.joblib')
breakdown = joblib.load('models/rf_model.joblib')


c:\Users\Lenovo Thinkpad T480\Desktop\Internship\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [10:26:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\data\../common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  warnings.warn(smsg, UserWarning)


In [13]:
df=pd.read_csv('data/data.csv').drop('Unnamed: 0',axis=1)
df.head()

,310A_FI_4303,310A_DI_3302,310A_PI_0316,310A_PI_0325,310A_PI_0578,310A_PI_0580,310A_FI_4301,310ASP01DI01SPM,310ASP01SI01SPM,310A_TI_5303_D,310A_TI_5304_D,310A_PDI_0308,timestamp
0,447,1.027,0.09,3.05,0.34,1.70,114,48,64,40,66,67,2025-06-18 11:47:10
1,215,1.027,0.10,2.77,0.46,2.69,0,48,59,41,73,68,2025-06-18 13:47:10
2,344,1.029,0.17,3.18,0.57,3.65,7,49,63,42,77,72,2025-06-18 15:47:10
3,298,1.073,0.98,5.02,0.63,3.51,0,59,56,72,70,75,2025-06-18 17:47:10
4,141,1.108,1.08,5.59,0.50,3.07,1,59,55,79,55,70,2025-06-18 19:47:10


In [18]:
half = df[len(df)//2:]
half.head()

,310A_FI_4303,310A_DI_3302,310A_PI_0316,310A_PI_0325,310A_PI_0578,310A_PI_0580,310A_FI_4301,310ASP01DI01SPM,310ASP01SI01SPM,310A_TI_5303_D,310A_TI_5304_D,310A_PDI_0308,timestamp
180,582,1.018,1.32,2.92,0.18,2.70,154,71,58,77,30,32,2025-07-03 11:47:10
181,592,1.059,1.40,5.54,0.20,3.74,133,71,58,84,31,35,2025-07-03 13:47:10
182,232,1.069,1.05,6.27,0.22,3.51,83,63,58,84,33,33,2025-07-03 15:47:10
183,219,1.046,1.13,6.47,0.23,3.29,16,64,58,85,34,34,2025-07-03 17:47:10
184,423,1.081,1.33,6.51,0.25,3.06,-1,68,58,87,35,36,2025-07-03 19:47:10


In [9]:
forecast

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [26]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

target_columns = half.columns
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(half)
scaled_df = pd.DataFrame(scaled_data, columns=half.columns)

Pred = {}
y_pred_unscaled = {}

for i, target_col in enumerate(target_columns):
    lagged_columns = []
    for col in half.columns:
        for j in range(1, 11):
            lagged = scaled_df[col].shift(j)
            lagged.name = f'{col}_lag{j}'
            lagged_columns.append(lagged)

    lagged_df = pd.concat(lagged_columns, axis=1)
    lagged_df['target'] = scaled_df[target_col]
    lagged_df = lagged_df.dropna()

    X = lagged_df.drop(columns='target')
    y = lagged_df['target']

    y_pred = forecast.predict(X)
    y_pred_temp = y_pred.reshape(-1, 1)

    dummy_pred = np.zeros((len(y_pred_temp), scaled_df.shape[1]))  # Correct shape
    dummy_pred[:, i] = y_pred_temp.flatten()

    #y_pred_unscaled[target_col] = scaler.inverse_transform(dummy_pred)[:, i]
    Pred[target_col] = dummy_pred[:,i]


In [31]:
for elt in Pred:
    scaler.inverse_transform(Pred[elt])

ValueError: Expected 2D array, got 1D array instead:
array=[1.01279616 0.68600464 0.58827698 0.5650965  0.54626983 0.55336136
 0.54569924 0.54589981 0.54009986 0.64975423 1.0119189  1.01102293
 1.00705242 1.0005697  1.00164151 1.00111222 0.9456473  0.94723874
 1.02467084 1.02987075 1.0275352  1.02725768 1.02725768 1.02725768
 1.02798176 0.94320571 0.92873597 0.93261528 0.9439919  0.9411267
 1.02161992 1.0270561  1.0246582  1.02487111 1.02467585 1.02386761
 1.02524042 0.89689118 0.85325462 0.578632   0.53955114 0.54269975
 0.54720086 0.58993143 0.72994643 0.50509661 0.60040295 0.55190164
 0.55765879 0.62370187 0.54679334 0.56185257 0.53839821 0.01304984
 0.70115596 0.55282193 0.56579328 0.70883852 1.01014519 0.99772012
 0.97456837 0.64153361 0.5727545  0.54102844 0.53885484 0.53702992
 0.70679998 0.7017293  0.68006724 0.78241587 1.01819253 0.99612588
 1.00265181 0.63998222 0.55000252 0.53721249 0.53358805 0.53085136
 0.69456547 0.51895839 0.52863103 0.58918762 0.67671925 0.99047917
 0.9

In [37]:
df00= pd.DataFrame(Pred)
df00.head()

,310A_FI_4303,310A_DI_3302,310A_PI_0316,310A_PI_0325,310A_PI_0578,310A_PI_0580,310A_FI_4301,310ASP01DI01SPM,310ASP01SI01SPM,310A_TI_5303_D,310A_TI_5304_D,310A_PDI_0308
0,1.012796,1.012796,1.012796,1.012796,1.012796,1.012796,1.012796,1.012796,1.012796,1.012796,1.012796,1.012796
1,0.686005,0.686005,0.686005,0.686005,0.686005,0.686005,0.686005,0.686005,0.686005,0.686005,0.686005,0.686005
2,0.588277,0.588277,0.588277,0.588277,0.588277,0.588277,0.588277,0.588277,0.588277,0.588277,0.588277,0.588277
3,0.565096,0.565096,0.565096,0.565096,0.565096,0.565096,0.565096,0.565096,0.565096,0.565096,0.565096,0.565096
4,0.546270,0.546270,0.546270,0.546270,0.546270,0.546270,0.546270,0.546270,0.546270,0.546270,0.546270,0.546270


In [39]:
scaler.inverse_transform(df00)

array([[1086.79426432,    1.5939971 ,    1.92431271, ...,   92.16445088,
          75.95971227,   60.78056598],
       [ 734.51300049,    1.21099744,    1.30340881, ...,   62.42642212,
          51.4503479 ,   40.84628296],
       [ 629.16258693,    1.09646062,    1.11772627, ...,   53.53320539,
          44.12077367,   34.88489592],
       ...,
       [ 989.89746714,    1.48865105,    1.75352986, ...,   83.98485112,
          69.21828389,   55.29753757],
       [1006.57107306,    1.50677857,    1.78291748, ...,   85.39236331,
          70.37832141,   56.24103475],
       [ 763.97247148,    1.24302573,    1.35533181, ...,   64.91326058,
          53.49994004,   42.51328456]])

In [4]:
def XGBoost_forecast(df, models, target_columns, lag=10):
        
    target_columns = df.columns
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df)
    scaled_df = pd.DataFrame(scaled_data, columns=df.columns)

    Pred = {}
    y_pred_unscaled = {}

    for i, target_col in enumerate(target_columns):
        lagged_columns = []
        for col in df.columns:
            for j in range(1, 11):
                lagged = scaled_df[col].shift(j)
                lagged.name = f'{col}_lag{j}'
                lagged_columns.append(lagged)

        lagged_df = pd.concat(lagged_columns, axis=1)
        lagged_df['target'] = scaled_df[target_col]
        lagged_df = lagged_df.dropna()

        X = lagged_df.drop(columns='target')
        y = lagged_df['target']

        y_pred = models.predict(X)
        y_pred_temp = y_pred.reshape(-1, 1)

        dummy_pred = np.zeros((len(y_pred_temp), scaled_df.shape[1]))  # Correct shape
        dummy_pred[:, i] = y_pred_temp.flatten()
        #y_pred_unscaled[target_col] = scaler.inverse_transform(dummy_pred)[:, i]
        Pred[target_col] = dummy_pred[:,i]
    return Pred

'310A_DI_3302'

In [7]:
from sklearn.preprocessing import MinMaxScaler
df = pd.read_csv('data/data.csv')
forecast=joblib.load('models/Forecast/310A_DI_3302.joblib')
df.drop(columns=['timestamp','Unnamed: 0'],inplace=True)
Pred = XGBoost_forecast(df,forecast,df.columns)

AttributeError: 'str' object has no attribute 'predict'

In [60]:
Pred

,310A_FI_4303,310A_DI_3302,310A_PI_0316,310A_PI_0325,310A_PI_0578,310A_PI_0580,310A_FI_4301,310ASP01DI01SPM,310ASP01SI01SPM,310A_TI_5303_D,310A_TI_5304_D,310A_PDI_0308
0,728.935725,1.030943,1.229785,3.503671,2.029642,2.156764,558.057913,63.438859,70.971233,48.978449,46.848951,43.719453
1,751.909199,1.050341,1.268019,3.612911,2.093365,2.224129,575.470085,63.918852,71.467778,50.501186,48.305482,45.109779
2,759.022184,1.056347,1.279857,3.646734,2.113095,2.244986,580.861194,64.067466,71.621517,50.972652,48.756450,45.540247
3,734.949956,1.036021,1.239794,3.532269,2.046324,2.174399,562.616249,63.564516,71.101224,49.377086,47.230257,44.083427
4,740.884599,1.041032,1.249671,3.560489,2.062785,2.191801,567.114264,63.688511,71.229494,49.770449,47.606516,44.442584
...,...,...,...,...,...,...,...,...,...,...,...,...
345,926.084204,1.197411,1.557892,4.441121,2.576487,2.734858,707.481688,67.557955,75.232368,62.045927,59.348278,55.650629
346,864.474989,1.145390,1.455358,4.148166,2.405597,2.554203,660.786519,66.270731,73.900756,57.962319,55.442218,51.922117
347,1359.794658,1.563628,2.279701,6.503433,3.779502,4.006617,1036.201715,76.619629,84.606513,90.793306,86.845771,81.898236
348,1377.782500,1.578816,2.309638,6.588966,3.829397,4.059362,1049.835151,76.995456,84.995299,91.985584,87.986210,82.986837


In [79]:
def load_data(path):
    df = pd.read_csv(path, parse_dates=['timestamp'])  # Ensure 'timestamp' column exists
    df.sort_values('timestamp', inplace=True)
    df.drop(columns='Unnamed: 0', inplace=True)

    return df
def get_current_slice(df, start_time, interval='100h'):
    end_time = start_time + pd.Timedelta(interval)
    return df[(df['timestamp'] >= start_time) & (df['timestamp'] < end_time)]

In [80]:
data = load_data('data/data.csv')
simulation_time = data['timestamp'].min()
slice_df = get_current_slice(data, simulation_time)

# Prepare response

target_columns = slice_df.drop(columns= 'timestamp').columns
prediction_data = slice_df.drop(columns=['timestamp'])
prediction_data

,310A_FI_4303,310A_DI_3302,310A_PI_0316,310A_PI_0325,310A_PI_0578,310A_PI_0580,310A_FI_4301,310ASP01DI01SPM,310ASP01SI01SPM,310A_TI_5303_D,310A_TI_5304_D,310A_PDI_0308
0,447,1.027,0.09,3.05,0.34,1.70,114,48,64,40,66,67
1,215,1.027,0.10,2.77,0.46,2.69,0,48,59,41,73,68
2,344,1.029,0.17,3.18,0.57,3.65,7,49,63,42,77,72
3,298,1.073,0.98,5.02,0.63,3.51,0,59,56,72,70,75
4,141,1.108,1.08,5.59,0.50,3.07,1,59,55,79,55,70
5,206,1.038,0.42,5.69,0.32,2.26,2,55,55,60,45,42
6,275,1.066,1.16,5.72,0.30,1.46,1,62,55,72,37,71
7,207,1.027,0.19,5.77,0.16,0.65,-1,54,55,56,34,44
8,-1,1.026,0.03,0.69,0.25,0.61,0,53,55,39,33,57
9,-1,1.026,0.05,0.15,0.34,1.77,-1,53,55,35,32,57


In [81]:
y_pred_forecast=XGBoost_forecast(prediction_data,forecast,target_columns)

In [82]:
y_pred_forecast

{'310A_FI_4303': array([0.63164145, 0.52445525, 0.54837883, 0.53173959, 0.73279649,
        0.54547155, 0.53121775, 0.57991165, 0.54138929, 0.53939587,
        0.57047236, 0.51607847, 0.52386588, 0.52991027, 0.53839153,
        0.03157161, 0.59311897, 0.9453578 , 0.94991249, 0.95146739,
        0.52981609, 0.6520586 , 0.52673763, 0.56722128, 0.59744   ,
        0.55083412, 0.54496062, 0.55389613, 0.10998432, 0.57700807,
        0.572088  , 0.56537414, 0.66029376, 0.56341326, 0.55659032,
        0.55419445, 0.54776591, 0.55033869, 0.54695714, 0.54882181]),
 '310A_DI_3302': array([0.63164145, 0.52445525, 0.54837883, 0.53173959, 0.73279649,
        0.54547155, 0.53121775, 0.57991165, 0.54138929, 0.53939587,
        0.57047236, 0.51607847, 0.52386588, 0.52991027, 0.53839153,
        0.03157161, 0.59311897, 0.9453578 , 0.94991249, 0.95146739,
        0.52981609, 0.6520586 , 0.52673763, 0.56722128, 0.59744   ,
        0.55083412, 0.54496062, 0.55389613, 0.10998432, 0.57700807,
        0.5720

In [2]:
import joblib
bd = joblib.load('models/rf_model.joblib')

In [18]:
import pandas as pd
df = pd.read_csv('data/Final.csv')
df.drop(columns=['Unnamed: 0','timestamp','faulty_SP','faulty_TK','faulty_VP'],inplace=True)


In [19]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df)
df = scaler.transform(df)



In [20]:
df.shape


(360, 18)

In [21]:
y_pred = bd.predict(df)

In [22]:
y_pred

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       ...,
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])